In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
energy_train = pd.read_parquet('energy_train.parquet')
energy_test2 = pd.read_parquet('energy_test2.parquet')
energy_test1 = pd.read_parquet('energy_test1.parquet')
forecasts = pd.read_parquet('forecasts.parquet')

In [ ]:
energy_train.info()

In [ ]:
energy_train.nunique()

In [ ]:
energy_test2.info()

In [ ]:
energy_test2.nunique()

In [ ]:
energy_test1.info()

In [ ]:
energy_test1.nunique()

## Aufgabe 1

### Aufgabe 1.1

Bei Trainingsdaten gibt es insgesamt 19968 Datenpunkte.        
Auf dem Test 1 gibt es: 7245 Datenpunkte und,      
Auf dem Test 2 gibt es: 7465 Datenpunkte.       
Insgesamt gibt es 14710 Datenpunkte auf den Testdaten.  

### Aufgabe 1.2

In [ ]:
tage = ['2020-11-11', '2021-07-21'] # Beispieltage 
tage_data = energy_train[energy_train['dtm'].dt.strftime('%Y-%m-%d').isin(tage)].copy()  #auf Tage filtern
tage_data.loc[:,'hour'] = tage_data['dtm'].dt.hour #Stunden extrahieren
tage_data.loc[:,'month'] = tage_data['dtm'].dt.month 

#### Tagesverlauf der Stromerzeugung

In [ ]:
# Visualisierung
fig = px.line(tage_data, x='hour', y='Solar_MWh', 
    color=tage_data['dtm'].dt.strftime('%Y-%m-%d'), 
    markers=True,
    labels={'hour': 'Uhrzeit', 'Solar_MWh': 'Erzeugte Menge an Solarstrom in MWh'},
    title="Tagesverlauf der Stromerzeugung für den 11.11.2020 und den 21.07.2021"
)
fig.show()

Die rote Kurve stellt die Stromerzeugung in MWh von dem 21.07.2021 dar, während die blaue Kurve die Stromerzeugung in MWH vom 11.11.2020 zeigt. 

Dabei handelt es sich um einen Sommertag (Juli) und um einen Wintertag (Dezember). Es ist deutlich zu erkennen, dass an dem Sommertag die Sonne bereits um kurz vor 4:00 Uhr aufgeht, was zur frühren Erzeugung an Solarstrom führt. Solarstrom wird auch fast bis 20:00 Uhr erzeugt. Die maximal Erzeugte Menge an Solarstrom wurde um 11:00 Uhr erreicht. 

Im Winter wurde hingegen erst ab etwa 7:00 Uhr Solartrom erzeugt und die höchste Menge an Solarstrom wurde um 12:00 Uhr erreicht. Danach sank die Kurve an erzeugter Menge an Solarstrom im MWh sehr schnell ab und es konnte nur bis kurz vor 16:00 Uhr Solarstrom erzeugt werden, da es danach höchstwahrscheinlich bereits dunkel war.

### Aufgabe 1.3

 #### Gesamtverlauf der Stromerzeugung

In [ ]:
fig = px.line(energy_train, x= 'dtm', y='Solar_MWh', 
              labels= {'dtm': 'Datum', 'Solar_MWh': 'Erzeugte Menge an Solarstrom in MWh'},
              title= 'Visualisierung des Gesamtverlaufes der Stromerzeugung (Trainingsdaten)')
fig.show()

In [ ]:
#Daten nach Monaten gruppieren und die Summe der Stromerzeugung berechnen
df_monthly = energy_train.groupby(energy_train['dtm'].dt.to_period('M'))['Solar_MWh'].sum().reset_index()
# Monatsperioden in Zeitstempel umwandeln, um sie für die Visualisierung zu nutzen
df_monthly['dtm'] = df_monthly['dtm'].dt.to_timestamp() 

# Visualisierung des Gesamtverlaufes der Stromerzeugung nach Monaten
fig = px.line(
    df_monthly,
    x='dtm',
    y='Solar_MWh',
    title='Visualisierung des Gesamtverlaufes der Stromerzeugung (Trainingsdaten)',
    labels={'dtm': 'Datum', 'Solar_MWh': 'Erzeugte Menge an Solarstrom in MWh'},
    markers=True
)

fig.show()

Die Kurve stellt den Verlauf der monatlichen Solarstromerzeugung in MWh über einen Zeitraum von zwei Jahren dar.

Es lässt sich ein deutlich saisonales Muster erkennen, das durch die jahreszeitlichen Schwankungen der Sonneneinstrahlung geprägt ist.

Im Sommer, insbesondere in den Monaten Juni und Juli, sind die Tage länger und die Sonne scheint intensiver. Dies führt dazu, dass die Solaranlagen mehr Strom erzeugen können. Die höchsten Werte der Stromerzeugung wurden in diesen Monaten erreicht, da die Sonne früher aufgeht, später untergeht und die Sonneneinstrahlung intensiver ist.

Im Winter hingegen, insbesondere in den Monaten Dezember und Januar, sind die Tage kürzer, und die Sonneneinstrahlung ist schwächer. Die Solarstromerzeugung beginnt später am Tag, erreicht geringere Höchstwerte und endet früher. Das zeigt sich in den deutlich geringeren Werten der Stromerzeugung in den Wintermonaten.

## Aufgabe 2

### Aufgabe 2.1

#### Wettermodelle zusammenführen

Zuerst auf Spaltennamen filtern und umbenennen auf das jeweilige Wettermodell, welches uns die weitere Verarbeitung vereinfacht 

In [ ]:
for model in ['DWD ICON', 'NCEP GFS']:
    # Filter für das Wettermodell
    forecasts_model = forecasts[forecasts['Weather Model'] == model].copy()
    
    # Spalten umbenennen
    forecasts_model = forecasts_model.rename(columns={
        'SolarDownwardRadiation': f'SolarDownwardRadiation_{model.replace(" ", "_")}',
        'CloudCover': f'CloudCover_{model.replace(" ", "_")}',
        'Temperature': f'Temperature_{model.replace(" ", "_")}'
    })
    
    # 'valid_datetime' berechnen
    forecasts_model['valid_datetime'] = pd.to_datetime(forecasts_model['ref_datetime']) + pd.to_timedelta(forecasts_model['valid_time'], unit='h')
    # muss so berechnet werden, da die Spalte ref_datetime den Zeitpunkt, an der die Wettervorhersage veröffentlicht wurde ist. valid_time gibt an, 
    # für wie viele Stunden die Wettervorhersage gültig war 
    # tatsächliche Gültigkeitsdauer: ref_datetime + timedelta(hours = valid_time)
    # -> timedata, um Zeitspannen zu erzeugen , unit = 'h', da wir in Stunden rechnen

    # Speicherung der Daten für die einzelnen Modelle
    if model == 'DWD ICON':
        forecasts_dwd = forecasts_model
    else:
        forecasts_ncep = forecasts_model

# Zusammenführen der beiden Modelle
forecasts_combined = pd.merge(forecasts_ncep, forecasts_dwd, on=['ref_datetime', 'valid_time', 'valid_datetime'], how='inner')

#### Energiedaten und Wettervorhersagen zusammenführen

um alle wichtige Informationen in einem Df zu haben und um weitere Vorverarbeitungsschritte durchzuführen (auch für Aufgabe 3 relevant)

In [ ]:
energy_train_mit_forecast = pd.merge(energy_train, forecasts_combined, left_on=['dtm', 'ref_datetime'], right_on=['valid_datetime', 'ref_datetime'], how='inner')
energy_train_mit_forecast

In [ ]:
energy_train_mit_forecast['Monat'] = energy_train_mit_forecast['dtm'].dt.strftime('%b')  # Monaten extrahieren
energy_train_mit_forecast['Zeit'] = energy_train_mit_forecast['dtm'].dt.hour #Stunden extrahieren

features = ['Monat', 'SolarDownwardRadiation_NCEP_GFS', 'CloudCover_NCEP_GFS', 'Temperature_NCEP_GFS', 'Zeit'] #Attributen der Wettervorhersage
energy_train_mit_forecast['Zeit'] = energy_train_mit_forecast['Zeit'].astype(str)

#### Welche Attribute haben einen Zusammenhang mit der Stromerzeugung? (Scatterplot)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20,12))
fig.tight_layout()

for i, col in enumerate(features):
    #plt.subplot(4, 3, i+1)
    energy_train_mit_forecast[["Solar_MWh",col]].plot.scatter(x=col, y="Solar_MWh", ax=axes[i//3,i%3], alpha=0.2)

#### Solar_MWh vs. SolarDownwardRadiation
Ein sehr starker linearer Zusammenhang ist zu erkennen: Mit zunehmender Sonneneinstrahlung steigt die Solarstromproduktion stark an.

#### Solar_MWh vs. Temperature:
Höhere Temperaturen fördern zwar die Stromproduktion, aber der Effekt ist nicht so stark wie bei der Sonneneinstrahlung. Dies könnte auf eine indirekte Beziehung hindeuten, z. B. dass wärmere Tage auch sonniger sind.

#### Solar_MWh vs. Zeit:

- Die Solarstromproduktion zeigt einen starken Zusammenhang mit der Tageszeit.
- Der Plot verdeutlicht, dass der Strom hauptsächlich zwischen den Stunden 6:00 und 17:00 erzeugt wird, mit einem deutlichen Spitzenwert zwischen 10:00 und 13:00. Dies entspricht dem Sonnenhöchststand, bei dem die Sonneneinstrahlung am intensivsten ist.
- Außerhalb dieser Stunden ist die Stromproduktion nahezu null, was den erwarteten Zusammenhang mit der Verfügbarkeit von Sonnenlicht bestätigt.

#### Solar_MWh vs. Monat:

Es gibt eine klare saisonale Variation:
- Die Solarstromproduktion ist in den Sommermonaten (April bis August) am höchsten. Dies ist auf die längeren Tagesstunden und intensivere Sonneneinstrahlung in diesen Monaten zurückzuführen.
- In den Wintermonaten (Oktober bis Februar) ist die Produktion deutlich niedriger, was durch kürzere Tage, tiefere Sonnenstände und häufigere Bewölkung erklärt werden kann.

#### Solar_MWh vs. CloudCover
Kein klarer Zusammenhang zwischen Bewölkungsanteil (CloudCover) und Solarstromproduktion ist zu erkennen.

#### Korrelationsmatrix der Wetterattribute und Solarstrom

In [ ]:
corr_matrix = energy_train_mit_forecast[['SolarDownwardRadiation_NCEP_GFS', 'CloudCover_NCEP_GFS', 'Temperature_NCEP_GFS', 
                                         'SolarDownwardRadiation_DWD_ICON', 'CloudCover_DWD_ICON', 'Temperature_DWD_ICON',
                                         'Solar_MWh']].corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    labels={'color': 'Korrelationskoeffizient'},
    title='Korrelationsmatrix der Wetterattribute und Solarstrom'
)
fig.show()

### Aufgabe 2.2

1) Sonneneinstrahlung (SolarDownwardRadiation)
- Die Daten zeigen, dass Solar_MWh (erzeugte Strommenge) sehr stark von der Sonneneinstrahlung abhängt.
Korrelation: Sehr starke Korrelation (ca. 0.93). Dies ist die höchste beobachtete Korrelation, was bedeutet, dass je mehr Sonneneinstrahlung vorhanden ist, desto mehr Strom wird produziert.


 ## Aufgabe 3

In [ ]:
print('Spaltennamen im Df:', forecasts.columns)
print('Spaltennamen im Df:', energy_train.columns)

In [ ]:
# Merging mit energy_train
energy_train['dtm'] = pd.to_datetime(energy_train['dtm'])
energy_train['ref_datetime'] = pd.to_datetime(energy_train['ref_datetime'])
energy_train_mit_forecast = pd.merge(energy_train, forecasts_combined, left_on=['dtm', 'ref_datetime'], right_on=['valid_datetime', 'ref_datetime'], how='inner')

Entfernen von Datensätzen ohne Label -> Da Datensätze ohne Label (Solar_MWh) können nicht für das Training verwendet werden, denn sie enthalten keine Informationen über das Ziel, das vorhergesagt werden soll + damit nachher keine unterschiedliche Verteilung vorhanden ist

In [ ]:
energy_train_mit_forecast = energy_train_mit_forecast[energy_train_mit_forecast['Solar_MWh'].isna()== False]

### Erstellen von Trainings- und Testdaten

Man muss jetzt in Trainings- und Testdaten aufteilen, um Dataleaks bei den kommenden Vorverarbeitungsschritten zu vermeiden

In [ ]:
#Extrahieren von Monat und Jahr, bevor gefiltert wird, um eine zeitliche Struktur der Daten zu behalten 
energy_train_mit_forecast['month_year'] = energy_train_mit_forecast['dtm'].dt.to_period('M')

# Daten in Test- und Traingsdaten aufteilen, achtung auf Data leakage
df_train = energy_train_mit_forecast[(energy_train_mit_forecast['month_year'] >= '2020-09') & (energy_train_mit_forecast['month_year'] <= '2022-06')]
df_test = energy_train_mit_forecast[energy_train_mit_forecast['month_year'] > '2022-06']

### Datenanalyse von den Daten (zuerst kathegorischen, dann numerischen)

In [ ]:
numeric_columns = df_train.select_dtypes(include=np.number).columns
categorical_columns = df_train.select_dtypes(exclude=np.number).columns

df_train[categorical_columns].describe()

In [ ]:
drop_columns = {'dtm', 'ref_datetime','valid_time', 'valid_datetime', 'month', 'year', 'Weather Model_x', 'Weather Model_y'}
df_train = df_train.drop(drop_columns, axis=1, errors='ignore')
df_test = df_test.drop(drop_columns, axis=1, errors='ignore')

In [ ]:
df_train.describe(exclude=np.number)

In [ ]:
interesting_columns = ['Solar_capacity_mwp', 'Solar_MWh', 'SolarDownwardRadiation_DWD_ICON', 'CloudCover_DWD_ICON',	'Temperature_DWD_ICON',	'SolarDownwardRadiation_NCEP_GFS',	'CloudCover_NCEP_GFS',	'Temperature_NCEP_GFS']

### Untersuchung von Ausreißern

Untersuchung von Ausreißern, da sie  das Modell negativ beeinflussen können (vor allem wenn man ein lineares Modell trainiert) 
-> so bleibt die Modellleistung stabiler und generalisierbarer

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(10,10))
fig.tight_layout()

for i, col in enumerate(interesting_columns):
    plt.subplot(2, 4, i+1)
    df_train[col].plot(kind="box")

### Datenbereiningung

Entfernen von Ausreißern (kleiner als 0.5% Quantil und größer als 99.5% Quantil Punkte entfernen )

In [ ]:
display(df_train[interesting_columns].quantile(0.995))
display(df_train[interesting_columns].quantile(0.005))

In [ ]:
upper_limits = df_train[interesting_columns].quantile(0.995)
lower_limits = df_train[interesting_columns].quantile(0.005)

In [ ]:
df_train_reduced = df_train.copy()
df_test_reduced = df_test.copy()

In [ ]:
# Für jede Spalte behalten wir: Daten die < (99.5%-Quantil) sind und > (0.5%-Quantil) sind ODER die NaN sind 
for col in interesting_columns:
    df_train_reduced = df_train_reduced[((df_train_reduced[col] <= upper_limits[col]) & (df_train_reduced[col] >= lower_limits[col])) | df_train_reduced[col].isna()]
    df_test_reduced = df_test_reduced[((df_test_reduced[col] <= upper_limits[col]) & (df_test_reduced[col] >= lower_limits[col])) | df_test_reduced[col].isna()]

df_train_reduced.describe(include=np.number)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(10,10))
fig.tight_layout()

for i, col in enumerate(interesting_columns):
    plt.subplot(2, 4, i+1)
    df_train_reduced[col].plot(kind="box")

### Behandlung von fehlenden Datensätzen

In [ ]:
df_train_reduced.isna().sum() # isna() wandelt jeden Wert in einen Boolschen Datentyp (0/1) um. sum() summiert diese Werte pro Spalte. Man erhält also die Anzahl der NaN Werte pro Spalte.
# SOlarDownwardRadiation hat viele Na Werte

Damit nicht viele Zeilen aufgrund von NaN-Werten verloren geht, benutzt man SimpleImputer um sicher zu stellen, dass die Daten vollständig sind und trotzdem möglichst wenig Informationsverlust entsteht. Mittelwert verwenden man für numerische Werte, den häufigsten Wert benutzt man für kategorische Daten.

In [ ]:
categorical_columns = df_train_reduced.select_dtypes(exclude=np.number).columns
numeric_columns = df_train_reduced.select_dtypes(include=np.number).columns
# Für die Label-artigen Columns ersetzen wir die fehlenden Werte nicht
numeric_columns = numeric_columns.drop("Solar_MWh") 

In [ ]:
from sklearn.impute import SimpleImputer

imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

df_train_reduced.loc[:,categorical_columns] = imp_freq.fit_transform(df_train_reduced[categorical_columns])
df_train_reduced.loc[:,numeric_columns] = imp_mean.fit_transform(df_train_reduced[numeric_columns])

df_test_reduced.loc[:,categorical_columns] = imp_freq.transform(df_test_reduced[categorical_columns])
df_test_reduced.loc[:,numeric_columns] = imp_mean.transform(df_test_reduced[numeric_columns])

### Feature Engineering

Das Label (Solar_MWh) wird aus den Features entfernt, da es nicht als Eingabe fürs Training genutzt werden darf. Es bleibt separat in y_train und y_test.

In [ ]:
df_train = df_train.drop("Solar_MWh", axis=1, errors="ignore")

In [ ]:
# Die Label-Spalte speichern wir in der Variable y_train bzw. y_test
y_train = df_train_reduced["Solar_MWh"]
y_test = df_test_reduced["Solar_MWh"]

# Nun werden die Spalten endgültig entfernt
df_train_reduced = df_train_reduced.drop(["Solar_MWh", "dtm"], axis=1, errors="ignore")
df_test_reduced = df_test_reduced.drop(["Solar_MWh", "dtm"], axis=1, errors="ignore")

### OneHotEncoding

Kategoriale Variablen müssen in numerische Werte umgewandelt werden. Benutzen dazu OneHotEncoding, da es die Reihenfolge vermeidet

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="if_binary")

categorical_columns = df_train_reduced.select_dtypes(exclude=np.number).columns

one_hot_columns_train = pd.DataFrame(enc.fit_transform(df_train_reduced[categorical_columns]))
one_hot_columns_train.columns = enc.get_feature_names_out()
one_hot_columns_train.index = df_train_reduced.index

one_hot_columns_test = pd.DataFrame(enc.transform(df_test_reduced[categorical_columns]))
one_hot_columns_test.columns = enc.get_feature_names_out()
one_hot_columns_test.index = df_test_reduced.index

Zusammenfügen der finalen Trainings- und Testdaten, um endgültige Datensätze (X_train und X_test) für das Training und die Validierung zu haben

In [ ]:
numeric_columns = df_train_reduced.select_dtypes(include=np.number).columns

X_train = pd.concat([df_train_reduced[numeric_columns], one_hot_columns_train], axis=1)
X_test = pd.concat([df_test_reduced[numeric_columns], one_hot_columns_test], axis=1)

In [ ]:
X_train 

Warum wurde diese Reihenfolge gewählt?

Zuerst Datenbereinigung und Vorverarbeitung gemacht, um unvollständige oder inkonsistente Daten zu bereinigen. 
Danach Spaltenauswahl und Transformationen, um Datenkonsistenz zu behalten und Modellkompatibilität zu garantieren.
Anschließend wurde zur Vermeidung von Datenleaks, das Dataframe Energy_train_mit_forecast gesplitet.
Zum Schluss Feature Engineering und Encoding, zur Optimierung für maschinelle Lernalgorithmen.

## Aufgabe 4

Potentielle Reduktion der Solarstrahlung durch Bewölkung

In [ ]:
energy_train_mit_forecast['CloudRadiationLoss'] =(energy_train_mit_forecast['SolarDownwardRadiation_DWD_ICON']* energy_train_mit_forecast['CloudCover_DWD_ICON'])

In [ ]:
fig = px.scatter(energy_train_mit_forecast, x='CloudRadiationLoss', y='Solar_MWh', trendline='ols', title='Zusammenhang zwischen der potentiellen Reduktion der Solarstrahlung durch Bewölkung und der tatsächlichen Solarerzeugung', 
                 labels={'CloudRadiationLoss': 'potentielle Reduktion der Solarstrahlung', 'Solar_MWh': 'Erzeugte Strommenge in MWh'},opacity=0.6)
fig.show()

In [ ]:
energy_train_mit_forecast['Relative_Reduktion'] = (
    energy_train_mit_forecast['CloudRadiationLoss'] / 
    energy_train_mit_forecast['SolarDownwardRadiation_DWD_ICON']
)

-> zeigt einen Zusammenhang zwischen potentiellen Reduktion der Solarstrahlung durch Bewölkung und der erzeugten Strommenge

-> Bewölkung hat einen negativen Einfluss auf die erzeugte Strommenge

In [ ]:
fig = px.scatter(energy_train_mit_forecast, x='Relative_Reduktion', y='Solar_MWh', trendline='ols', title='', 
                 labels={'CloudRadiationLoss': 'potentielle Reduktion der Solarstrahlung', 'Solar_MWh': 'Erzeugte Strommenge in MWh'},opacity=0.6)
fig.show()

-> schwacher Zusammenhang und eine große Streuung -> es gibt andere Faktoren, die eine wichtigere Rolle bei der Stromerzeugung spielen

In [ ]:
energy_train_mit_forecast['effective_radiation_DWD_ICON'] = energy_train_mit_forecast['SolarDownwardRadiation_DWD_ICON'] * (1 - energy_train_mit_forecast['CloudCover_DWD_ICON'])
energy_train_mit_forecast['effective_radiation_NCEP_GFS'] = energy_train_mit_forecast['SolarDownwardRadiation_NCEP_GFS'] * (1 - energy_train_mit_forecast['CloudCover_NCEP_GFS'])
energy_train_mit_forecast['radiation_temp_interaction_DWD_ICON'] = energy_train_mit_forecast['SolarDownwardRadiation_DWD_ICON'] * energy_train_mit_forecast['Temperature_DWD_ICON'] 
energy_train_mit_forecast['radiation_temp_interaction_NCEP_GFS'] = energy_train_mit_forecast['SolarDownwardRadiation_NCEP_GFS'] * energy_train_mit_forecast['Temperature_NCEP_GFS'] 
energy_train_mit_forecast['temperature_squared_DWD_ICON'] = energy_train_mit_forecast['Temperature_DWD_ICON'] ** 2  
energy_train_mit_forecast['temperature_squared_NCEP_GFS'] = energy_train_mit_forecast['Temperature_NCEP_GFS'] ** 2  
energy_train_mit_forecast.info()

In [ ]:
energy_train_mit_forecast.head(2)

In [ ]:
energy_train_mit_forecast.columns

### Daten auf die Jahreszeiten aufteilen

In [ ]:
energy_train_mit_forecast['month'] = energy_train_mit_forecast['month_year'].dt.month # verweden um Monatsnummer zu erhalten

In [ ]:
# Funktion für die Jahreszeiten
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'

In [ ]:
# Jeweiligen Jahreszeiten Zuordnen

energy_train_mit_forecast['Season'] = energy_train_mit_forecast['month'].apply(get_season)

In [ ]:
fig = px.box(
    energy_train_mit_forecast,
    x='Season',
    y='Solar_MWh',
    title='Solarstromerzeugung nach Jahreszeiten',
    labels={'Season': 'Jahreszeit', 'Solar_MWh': 'Erzeugte Solarstrommenge (MWh)'}
)
fig.show()

### Schauen wann die Sonne jeden Tag auf- und untergeht

In [ ]:
# Extrahiere das Datum aus der `dtm`-Spalte
energy_train_mit_forecast['date'] = energy_train_mit_forecast['dtm'].dt.date

# Filtriere Zeilen mit Solar_MWh > 0
energy_positive = energy_train_mit_forecast[energy_train_mit_forecast['Solar_MWh'] > 0]

# Gruppiere nach `date` und finde den ersten Zeitpunkt pro Tag
first_positive_time = energy_positive.groupby('date')['dtm'].min().reset_index()

# Ergebnis anzeigen
# first_positive_time

In [ ]:
# Umbenennen der dtm-Spalte in `first_positive_dtm` im first_positive_time DataFrame
first_positive_time = first_positive_time.rename(columns={'dtm': 'first_positive_dtm'})

# Mergen von first_positive_time mit energy_train_mit_forecast
energy_train_mit_forecast = pd.merge(energy_train_mit_forecast, first_positive_time, on='date', how='left', suffixes=('', '_from_first_positive_time'))

In [ ]:
# Finde den letzten Zeitpunkt pro Tag, an dem Solar_MWh > 0 war
last_positive_time = energy_positive.groupby('date')['dtm'].max().reset_index()

# Funktion, um den ersten Zeitpunkt nach dem letzten positiven Wert zu finden, wo Solar_MWh == 0
def find_zero_after_last(df, last_times):
    results = []
    for _, row in last_times.iterrows():
        date = row['date']
        last_positive_dtm = row['dtm']
        # Filtriere Daten für das entsprechende Datum
        daily_data = df[df['date'] == date]
        # Finde den ersten Zeitpunkt nach dem letzten positiven Wert
        zero_after_last = daily_data[
            (daily_data['dtm'] > last_positive_dtm) & (daily_data['Solar_MWh'] == 0)
        ]
        if not zero_after_last.empty:
            results.append({'date': date, 'zero_after_last_dtm': zero_after_last.iloc[0]['dtm']})
        else:
            results.append({'date': date, 'zero_after_last_dtm': None})
    return pd.DataFrame(results)

# Finde Zeitpunkte für Solar_MWh == 0 nach dem letzten positiven Wert
zero_times = find_zero_after_last(energy_train_mit_forecast, last_positive_time)

# Ergebnis anzeigen
# print(zero_times)

In [ ]:
# Mergen von zero_times mit energy_train_mit_forecast auf 'date' Spalte
energy_train_mit_forecast = pd.merge(energy_train_mit_forecast, zero_times, on='date', how='left', suffixes=('', '_zero_after_last_dtm'))

In [ ]:
energy_train_mit_forecast.head()

### Herausfinden, wann die Sonne in den jeweiligen Jahreszeiten im durchschnitt auf- und untergeht

In [ ]:
# Funktion, um die Zeit seit Mitternacht in Sekunden zu berechnen
def time_since_midnight(dt_series):
    return dt_series.dt.hour * 3600 + dt_series.dt.minute * 60 + dt_series.dt.second

# Berechne die Sekunden seit Mitternacht
energy_train_mit_forecast['first_positive_seconds'] = time_since_midnight(energy_train_mit_forecast['first_positive_dtm'])
energy_train_mit_forecast['zero_after_seconds'] = time_since_midnight(energy_train_mit_forecast['zero_after_last_dtm'])

# Gruppiere nach Season und berechne den Mittelwert der Sekunden
average_times_by_season = (
    energy_train_mit_forecast
    .groupby('Season')[['first_positive_seconds', 'zero_after_seconds']]
    .mean()
    .reset_index()
)

# Konvertiere die Sekunden zurück in ein Zeitformat
average_times_by_season['avg_first_positive_time'] = pd.to_timedelta(average_times_by_season['first_positive_seconds'], unit='s')
average_times_by_season['avg_zero_after_time'] = pd.to_timedelta(average_times_by_season['zero_after_seconds'], unit='s')

# Nur relevante Spalten anzeigen
average_times_by_season = average_times_by_season[['Season', 'avg_first_positive_time', 'avg_zero_after_time']]

# Ergebnis anzeigen
average_times_by_season


## Schauen welche Features am aussagekräftigsten ist

In [ ]:
# Berechnung der Korrelationen
correlations = energy_train_mit_forecast.corr(numeric_only=True)['Solar_MWh'].sort_values(ascending=False)

# Ausgabe der stärksten Korrelationen
print("Korrelation mit Solar_MWh:")
print(correlations)

Fazit:

von den neu hinzugefüten Features kann man sehen, dass radiation_temp_interaction (auf beiden Wettervorhersagemodellen) den stärksten Einfluss auf Solar_MWh haben. -> Feature berücksichtigt die Effizienz von den Solaranlagen im Bezug zur Temperatur

effective_radiation (tatsächliche Menge an Strahlung, die die Solarpads erreicht, nach Abzug der durch Wolken verlorenen Strahlung) haben ebenfalls einen Einfluss auf Solar_MWh, jedoch nur einen mittleren Einfluss. Genauso wie CloudRadiationLoss , welches den Verlust an Strahlung durch Wolkenbedeckung repräsentiert.

temperature_squared und termperature je nach Wettermodell haben nur einen moderaten Einfluss auf Solar_MWh.

Es ist jedoch auffällig, dass das Feature SolarDownwardRadiation (die direkte Sonneneinstrahlung) den größten Einfluss hat. 